In [1]:
# Step 1: Imports and Setup

from typing import Sequence, Literal
from typing_extensions import TypedDict

from langchain import hub
from langchain_core.messages import BaseMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

from langgraph.graph import END, StateGraph, START
from langgraph.prebuilt import ToolNode
from langgraph.graph.message import add_messages

/Users/harshrathi/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Step 2: State Management

class AgentState(TypedDict):
    messages: Sequence[BaseMessage]
    cycle_count: int

In [3]:
# Step 3: Reflection Feedback Model

class ReflectionFeedback(BaseModel):
    feedback: str = Field(description="Critique and recommendations for improvement.")

In [4]:
# Step 4: Output Generation

def generate_output(state: AgentState) -> dict:
    """
    Generates the initial output based on the user's query.

    Args:
        state (AgentState): The current state.

    Returns:
        dict: Updated state with the generated response.
    """
    messages = state["messages"]
    user_query = messages[0].content

    model = ChatOpenAI(temperature=0.7, model="gpt-4")
    prompt = PromptTemplate(
        template="Respond to the following query in a clear and structured manner:\n\n{query}",
        input_variables=["query"],
    )
    llm_chain = prompt | model

    response = llm_chain.invoke({"query": user_query})
    return {
        "messages": state["messages"] + [HumanMessage(content=response)],
        "cycle_count": state.get("cycle_count", 0)
    }

In [5]:
# Step 5: Reflection Process

def reflect_output(state: AgentState) -> dict:
    """
    Reflects on the quality of the generated output.

    Args:
        state (AgentState): The current state.

    Returns:
        dict: Updated state with the reflection feedback.
    """
    messages = state["messages"]
    generated_output = messages[-1].content

    reflection_prompt = PromptTemplate(
        template="""You are an expert reviewer. Critique the following output based on structure, clarity, depth, and relevance. Provide specific recommendations for improvement.

Output:
{output}

Critique:""",
        input_variables=["output"],
    )

    model = ChatOpenAI(temperature=0.5, model="gpt-4")
    llm_chain = reflection_prompt | model.with_structured_output(ReflectionFeedback)

    reflection = llm_chain.invoke({"output": generated_output})
    return {
        "messages": state["messages"] + [HumanMessage(content=reflection.feedback)],
        "cycle_count": state.get("cycle_count", 0)
    }

In [6]:
# Step 6: Revision Process

def revise_output(state: AgentState) -> dict:
    """
    Revises the generated output based on the reflection feedback.

    Args:
        state (AgentState): The current state.

    Returns:
        dict: Updated state with the revised response.
    """
    messages = state["messages"]
    generated_output = messages[-2].content  # Original output
    feedback = messages[-1].content        # Reflection feedback
    cycle_count = state.get("cycle_count", 0)

    revision_prompt = PromptTemplate(
        template="""You are tasked with revising the following output based on the provided feedback.

Original Output:
{output}

Feedback:
{feedback}

Revised Output:""",
        input_variables=["output", "feedback"],
    )

    model = ChatOpenAI(temperature=0.5, model="gpt-4")
    llm_chain = revision_prompt | model

    revised_output = llm_chain.invoke({"output": generated_output, "feedback": feedback})
    return {
        "messages": state["messages"] + [HumanMessage(content=revised_output)],
        "cycle_count": cycle_count + 1
    }

In [10]:
# Step 6: Define the Workflow with Should Continue State

# Define the workflow
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("generate", generate_output)
workflow.add_node("reflect", reflect_output)
workflow.add_node("revise", revise_output)
workflow.add_node("should_continue", should_continue)  # New node

# Define edges
workflow.add_edge(START, "generate")
workflow.add_edge("generate", "reflect")
workflow.add_edge("reflect", "revise")
workflow.add_edge("revise", "should_continue")  # Updated edge to route through should_continue

# Define conditional edges from should_continue
workflow.add_conditional_edge(
    "should_continue",
    "generate",
    condition=lambda state: state["cycle_count"] < 3
)
workflow.add_conditional_edge(
    "should_continue",
    END,
    condition=lambda state: state["cycle_count"] >= 3
)

# Compile the graph
graph = workflow.compile()

AttributeError: 'StateGraph' object has no attribute 'add_conditional_edge'

In [11]:
# Step 7: Should Continue State

def should_continue(state: AgentState) -> dict:
    """
    Determines whether the workflow should continue based on the cycle count.

    Args:
        state (AgentState): The current state.

    Returns:
        dict: Updated state with the next node to transition to.
    """
    cycle_count = state.get("cycle_count", 0)
    if cycle_count < 3:
        next_node = "generate"
    else:
        next_node = END

    return {
        "next_node": next_node,
        "cycle_count": cycle_count
    }

In [8]:
# Step 8: Execution Example

from IPython.display import Image, display
import pprint

inputs = {
    "messages": [
        HumanMessage(content="Write an essay on the impact of social media on modern communication."),
    ],
    "cycle_count": 0
}

for output in graph.stream(inputs):
    for key, value in output.items():
        pprint.pprint(f"Output from node '{key}':")
        pprint.pprint("---")
        pprint.pprint(value, indent=2, width=80, depth=None)
    pprint.pprint("\n---\n")

NameError: name 'graph' is not defined

In [9]:
# Step 9: Visualization (Optional)

try:
    display(Image(graph.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    print("Visualization requires additional dependencies.")

Visualization requires additional dependencies.
